# The Reweighting Tutorial
So, you've done some enhanced sampling simulations where you applied a biasing potential to a certain collective variable. For whatever reason, you now want to quantify the free energy surface of the modeled event in a collective variable that is _not the biased CV_. Having applied an artificial bias to the simulations, you cannot simply calculate the probability density of this new CV and convert to free energy. Instead, each frame's weight in the probability density must be measured based on the weight that the frame received in the biased CV simulations. 

## Analysis Steps:
### 1) Run EMUS or WHAM to get stitching constants in the biased CV FE surfaces
Before any analysis of the new CV can be done, we need to perform a free energy analysis (using either EMUS or WHAM; suggested to use EMUS) of the biased CV space to obtain the "stitching" or "normalization" constant for each window in the baised CV space. These constants are important for reweighting the biased CV sampling into a new collective variable space. 

Running EMUS has been covered in a different tutorial (see https://mccullaghlab.okstate.edu/group-pages/tutorials/93-pages/402-emus). The code provided below will initialize a whole bunch of variables to be used throughout this tutorial as well as load in the results from EMUS, the biased CV data, and unbiased CV data. 

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import IO

stitching_constant_file = 'sample_stitching_constants.dat' # the stitching_constant_file has the same organization and manipulations performed on the z values as shwon in the EMUS Tutorial.
meta_file = 'meta_file.dat' # this file is the same file used in the EMUS analysis to obtain the stitching constants; format: location of biased CV file, CV Equilibrium Value, CV Spring Constant
nWindows = 40 # int object; corresponds to the number of (RE)US windows to be analyzed
start = 1 # int object; which production run does the analysis begin with.
end = 15 # int object; which production run does the analysis stop at. 
unbiased_data_files_naming = '/path/to/ubiased/cv/data/files/window%d/data_file_name.dat' # str object; global or local location of unbiased CV files 
xmin = -28.5
xmax = 21.0
xbins = 132
nIterations = 1000
x_axis_label = 'Unbiased CV'

T = 300. #units of Kelvin
k_B = 0.0019872036 # kcal K^-1 mol^-1
beta = 2*k_B*T

nWindows_range = range(nWindows) # assumes windows are numbered with zero-indexing
nProductions_range = range(start,end+1)

# ----------------------------------------
# LOAD IN STITCHING CONSTANT DATA FROM EMUS OR WHAM 
z = np.exp(-np.loadtxt(stitching_constant_file)[:,0]/(k_B*T)) # undoing the manipulations performed on the z values when saved to file in the EMUS Tutorial...

# ----------------------------------------
# LOAD IN DATA ASSOCIATED WITH BIASED WINDOW'S DATA, EQUILIB VALUE, AND SPRING CONSTANT
file_list = []
r0_k_list = [] 
with open(meta_file,'r') as f: 
    for line in f:
        if not line.startswith('#'):
            temp = line.split()
            file_list.append(temp[0])
            r0_k_list.append([float(temp[1]),float(temp[2])])

# ----------------------------------------
# LOAD IN BIASED AND UNBIASED CV DATA
# Creating the data structure that will be used for the remainder of the analysis...
# data will be a list of 2d arrays with shape nValues x 2; for each row, index 0 will be biased CV, index 1 will be unbiased CV; nValues is the number of frames for window i
data = ['' for i in nWindows_range]
for i in nWindows_range:
    print('loading window', i)
    # NOTE: FILE NAME HANDLING MIGHT BE DIFFERENT IN YOUR DIRECTORY TREE
    temp_unbiased_data = np.loadtxt(unbiased_data_files_naming%(i))     # NOTE: assumes files pointed at by unbiased_data_files_naming only contain the unbiased CV; creates a 1d array of length nValues (number of frames in production runs to be analyzed...) 
    temp_biased_data = np.loadtxt(file_list[i])[:,1]
    if len(temp_unbiased_data) != len(temp_biased_data):
        print('unbiased data file', i, 'has different number of values than the biased cv data from the respective window. This should not happen.', count, len(temp_unbiased_data))
        sys.exit()
    temp_data = np.c_[temp_biased_data,temp_unbiased_data]
    data[i] = temp_data


### 2) Histogram new, unbiased collective variable data, adding the reweighted counts to the respective histogram bin to calculate the probability density/free energy. 
Each frame's weight is added to a the respective bin associated with the new, unbiased CV value. The weight of a frame is calculated using the frame's biased CV value, the stitching constants (calculated from the last step), and the equilibrium constants used for each window of the biased simulations. The corrected weight of a frame from a biased simulation is the inverse of the sum of volume-corrected (if needed) Boltzmann weights from each biasing potential used in the set of analyzed simulations. 

$ w(t) = \frac{1}{nValues * BinWidth * VolumeCorrection(t) * \frac{1}{nWindows} \sum \limits_{i}^{nWindows} \frac{1}{Z_{i}} exp\left[-\frac{k_{i}}{2k_{B}T}(r(t) - r_{i})^{2}\right]} $

where $w(t)$ is the weight of a frame, t, $nValues$ is the total number of frames being analyzed, $BinWidth$ is the width of histogram bins used for the new CV histogram, $VolumeCorrection(t)$ is a term included to normalize the volume of the biased CV space and is a frame-dependent value, $nWindows$ is the number of windows being summed over, and the exponential within the sum is the Boltzmann weight of frame t's biased CV value ($r(t)$ relative to window i's equilibrium value, $r_{i}$, and biasing force constant, $k_{i}$) scaled by the window i's stitching constant. The $nValues$ and $BinWidth$ terms are used to convert a count into the associated probability density value. A volume correction term is needed when entropy of states is not uniform throughout the biased collective variable space (e.g. more volume at large distances than at close distances). The remaining terms in the denominators represents the average Boltzmann factor associated with the frame's biased CV value within all windows. 

A frame's weight is added to the respective bin in the new, unbiased CV histogram. 

In [ ]:
# ----------------------------------------
# PREPARING HISTOGRAM VARIABLES
delta_x = (xMax - xMin)/xBins
x_half_bins = np.array([xMin + delta_x*(i+0.5) for i in range(xBins)])
x_edges = np.array([xMin + delta_x*i for i in range(xBins+1)])

# ----------------------------------------
# HISTOGRAMMING UNBIASED CV DATA WITH REWEIGHTING FROM BIASED CV DATA
nValues_total = 0
x_total_fe_counts = np.zeros(xBins)
for i in nWindows_range:
    nValues = len(data[i])
    nValues_total += nValues
    
    x_window_counts = np.zeros(xBins)
    x_window_fe_counts = np.zeros(xBins)

    for j in range(nValues):
        # ----------------------------------------
        # HISTOGRAMMING UNBIASED CV DATA POINT
        x_index = int((data[i][j][1] - xMin)/delta_x)
        if x_index < 0 or x_index > xBins:
            print('!!! 0 > x_index >= xBins ...', data[i][j][0], x_index, i, 'Histogram bounds are not wide enough in the x-dimension. Job failed.')
            sys.exit()
        elif x_index == xBins:
            x_index = -1
    
        # ----------------------------------------
        # CALCULATING WEIGHT OF BIASED DATA POINT IN CONSIDERATION OF CURRENT WINDOW ONLY (USEFUL FOR PLOTTING UNSTITCHED FREE ENERGY SURFACES)
        w = np.exp((-beta*r0_k_data[i][1])*(data[i][j][0] - r0_k_data[i][0])**2)/z[i]     # exp((-k/2*k_B*T)(r-r0)^2)/z; no volume correction...
        #w = (data[i][j][0]**2)*np.exp((-beta*r0_k_data[i][1])*(data[i][j][0] - r0_k_data[i][0])**2)/z[i]     # r^2 * exp((-k/2*k_B*T)(r-r0)^2)/z;; volume correction for distance space
        x_window_counts[x_index] += 1
        x_window_fe_counts[x_index] += 1/w
    
        # ----------------------------------------
        # CALCULATING WEIGHT OF BIASED DATA POINT IN CONSIDERATION OF ALL WINDOWS (STITCHED FREE ENERGY SURFACE)
        w = 0
        for k in nWindows_range:
            w+= np.exp((-beta*r0_k_data[k][1])*(data[i][j][0] - r0_k_data[k][0])**2)/z[k]       # exp((-k/2*k_B*T)(r-r0)^2)/z; no volume correction...
            #w+= (data[i][j][0]**2)*np.exp((-beta*r0_k_data[k][1])*(data[i][j][0] - r0_k_data[k][0])**2)/z[k]       # r^2 * exp((-k/2*k_B*T)(r-r0)^2)/z; 
        w /= nWindows # <exp((-k/2*k_B*T)(r-r0)^2)/z>; average reciprocal boltzmann weight of data point in all possible windows;
        
        x_total_fe_counts[x_index] += 1/w
        
    # ----------------------------------------
    # SEE STEP 4 BELOW: PLOTTING
    # FINISHING ANALYSIS OF THE REWEIGHTED PROB. DENSITY OF EACH INDIVIDUAL WINDOW
    x_window_prob_density = x_window_counts/(nValues*delta_x)
    plt.figure(1)
    plt.plot(x_half_bins[:],x_window_prob_density[:],zorder=3)
    
    # ----------------------------------------
    # FINISHING ANALYSIS OF THE UNSTITCHED, REWEIGHTED FREE ENERGY OF EACH INDIVIDUAL WINDOW
    x_window_fe_counts = -kT*np.log(x_window_fe_counts/(nValues*delta_x))  # no volume correction
    #x_window_fe_counts = -kT*np.log(x_window_fe_counts/(nValues*delta_x*four_pi))
    plt.figure(2)
    plt.plot(x_half_bins[x_window_counts > 10.], x_window_fe_counts[x_window_counts > 10],zorder=3)
    
    print('Done with window', i)

### 3) Bootstrapping to calculate a lower bound for the error associated with the reweighting results. 
A bootstrapping analysis is used to estimate error for the new reweighted free energy surface. A number of bootstrapping iterations is performed where, during an iteration, $nValues$ of data is randomly pulled from the original collective variable data set. This new dataset is used to calculate the respective, fake free energy surface. The free energy surfaces from all bootstrapping iterations are then used to calculate the standard deviation of the sample data. This error analysis uses random sampling from a dataset with replacement, treating each data point within the original dataset as an independent, uncorrelated data point. Due to this assumption, the standard deviation calculated from bootstrapping represents the lower estimate for error in the reweighted free energy. 

In [ ]:
print('Beginning bootstrap analysis to approximate error in reweighted CVs')
original_data = np.empty((0,2))
for i in nWindows_range:
    original_data = np.concatenate((original_data,np.array(data[i])))

if original_data.shape != (nValues_total,2):
    print(original_data.shape, nValues_total, 'something went wrong during bootstrapping. The data structure does not have the expected shape.')
    sys.exit()

x_bootstrap_results = []
for i in range(nIterations):
    print('Starting Step %d of %d steps in bootstrap analysis'%(i,nIterations))
    
    # ----------------------------------------
    # CREATING SAMPLE DATA FOR BOOTSTRAP ITERATION
    sample_data = original_data[np.random.randint(nValues_total,size=nValues_total)]
    
    # ----------------------------------------
    # HISTOGRAMMING UNBIASED CV DATA POINT
    for j in range(nValues_total):
        x_index = int((sample_data[j,1] - xMin)/delta_x)
        if x_index == xBins:
            x_index = -1
        
        w = 0
        for k in nWindows_range:
            w+= np.exp((-beta*r0_k_data[k][1])*(sample_data[j,0] - r0_k_data[k][0])**2)/z[k]       # exp((-k/2*k_B*T)(r-r0)^2)/z; no volume correction...
        w /= nWindows # <exp((-k/2*k_B*T)(r-r0)^2)/z>; average reciprocal boltzmann weight of data point in all possible windows;
        x_total_fe_bootstrap[x_index] += 1/w
        x_total_fe_bootstrap = -kT*np.log(x_total_fe_bootstrap/(nValues*delta_x))  # no volume correction
        x_total_fe_bootstrap -= np.ndarray.min(x_total_fe_bootstrap)
        x_bootstrap_results.append(x_total_fe_bootstrap)
        
### NOTE: CALCS THE STANDARD DEVIATION OF THE BOOTSTRAPPED DATA
x_std_error = np.std(np.array(x_bootstrap_results),axis=0) # creates a 1d array of std dev values for each bin in the ubiased CV histogram 
np.savetxt('x_data_bootstrap_error.dat', x_std_error, fmt='%15.10f')


### 4) Perform plotting during the reweighting analysis for sanity checks. 
Plotting of probability densities and unstitched free energy graphs are useful to present the quality of sampling and stitching in the new CV space.

In [ ]:
# ----------------------------------------
# FINISHING PLOTTING OF THE REWEIGHTED PROB. DENSITY OF EACH INDIVIDUAL WINDOW - XDATA
IO.finish_plot(1,'reweighted_x_axis_prob_density.png',x_axis_label,'Probability Density',xlim=(xMin,xMax))

# ----------------------------------------
# FINISHING PLOTTING OF THE REWEIGHTED, UNSTITCHED FREE ENERGY - XDATA
IO.finish_plot(2,'reweighted_x_axis_unstitched_fe.png',x_axis_label,r'Relative Free Energy (kcal mol$^{-1}$)',xlim=(xMin,xMax))

# ----------------------------------------
# PLOTTING REWEIGHTED X-DATA FE SURFACE
x_total_fe_counts = -kT*np.log(x_total_fe_counts/(delta_x*nValues_total)) # no volume correction
np.savetxt('reweighted_x_axis_stitched_fe.dat', np.c_[range(xBins),x_half_bins,x_total_fe_counts], fmt='%15.10f')

plt.figure(3)
plt.errorbar(x_half_bins[:],x_total_fe_counts[:],yerr=x_std_error,ecolor='r',elinewidth=0.5,zorder=3)
IO.finish_plot(3, 'reweighted_x_axis_stitched_fe.png', x_axis_label, r'Relative Free Energy (kcal mol$^{-1}$)',xlim=(xMin,xMax),ylim=(-0.05,10)) # NOTE
plt.close()
